In [1]:
pip install webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Setup

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

### URLs a ser extraídos

In [3]:
# Lista de URLs dos filmes
filmes_urls = {
    "The Father": "https://www.imdb.com/title/tt10272386/reviews/",
    "Soul": "https://www.imdb.com/title/tt2948372/reviews/",
    "The Trial of the Chicago 7": "https://www.imdb.com/title/tt1070874/reviews/",
    "Bad Boys for Life": "https://www.imdb.com/title/tt1502397/reviews/",
    "Enola Holmes": "https://www.imdb.com/title/tt7846844/reviews/",
    "Homem-Aranha: Sem Volta para Casa": "https://www.imdb.com/title/tt10872600/reviews/",
    "Cruella": "https://www.imdb.com/title/tt3228774/reviews/",
    "Shang-Chi and the Legend of the Ten Rings": "https://www.imdb.com/title/tt9376612/reviews/",
    "A Quiet Place Part II": "https://www.imdb.com/title/tt8332922/reviews/",
    "The Black Phone": "https://www.imdb.com/title/tt7144666/reviews/"
}


In [4]:
# Configurar Chrome
options = Options()
# options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scraping_reviews(titulo, url):
    comentarios = []
    
    try:
        driver.get(url)
        time.sleep(2)

        # Clica no botão "All" para mostrar todas as reviews
        botoes = driver.find_elements(By.TAG_NAME, 'button')
        botoes_texto = [botao.text.strip() for botao in botoes]
        print(f"Botões encontrados: {botoes_texto}")

        botao_clicado = False
        for idx, botao in enumerate(botoes):
            if botao.text.strip() == "All":
                print(f"Botão 'All' encontrado na posição {idx}")
                driver.execute_script("arguments[0].scrollIntoView(true);", botao)
                time.sleep(1)
                botao.click()
                botao_clicado = True
                print(f"Botão 'All' clicado para {titulo}")
                time.sleep(3)
                break

        if not botao_clicado:
            print(f"Botão 'All' não encontrado para {titulo}, pode já ter tudo visível.")

        # Agora fazer scroll automático até ao fundo para carregar todas as reviews
        scrolls = 250  # Número máximo de scrolls para tentar carregar tudo
        last_height = driver.execute_script("return document.body.scrollHeight")

        for i in range(scrolls):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"⬇️ Scroll {i+1}/{scrolls} feito para baixo...")
            time.sleep(2)  # esperar carregar
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("✅ Sem mais conteúdo para carregar!")
                break
            last_height = new_height

        # Capturar todos os comentários reais
        artigos = driver.find_elements(By.CSS_SELECTOR, 'article.user-review-item')

        for artigo in artigos:
            # Título (pode sempre ser extraído)
            try:
                titulo_review = artigo.find_element(By.CSS_SELECTOR, 'h3.ipc-title__text').text.strip()
            except:
                titulo_review = ""

            # Corpo (pode não existir por ser spoiler)
            try:
                corpo_review = artigo.find_element(By.CSS_SELECTOR, 'div.ipc-html-content-inner-div').text.strip()
            except:
                try:
                    corpo_review = artigo.find_element(By.CSS_SELECTOR, 'div.text.show-more__control').text.strip()
                except:
                    corpo_review = ""  

            # Concatenar (prioriza corpo se existir)
            if corpo_review:
                texto_review = f"{titulo_review} - {corpo_review}" if titulo_review else corpo_review
            else:
                texto_review = titulo_review

            try:
                data_review = artigo.find_element(By.CSS_SELECTOR, 'li.review-date').text.strip()
            except:
                data_review = None

            # Guardar se houver pelo menos algum texto
            if texto_review:
                comentarios.append({
                    'Filme': titulo,
                    'Comentario': texto_review,
                    'Data': data_review
                })

    except Exception as e:
        print(f"Erro ao scrapear {titulo}: {e}")

    return comentarios

# Scraping para todos os filmes
todos_comentarios = []

for titulo, url in filmes_urls.items():
    print(f"A scrapear {titulo}...")
    comentarios = scraping_reviews(titulo, url)
    todos_comentarios.extend(comentarios)
    time.sleep(2)

# Fechar driver
driver.quit()

# Criar DataFrame
df_reviews = pd.DataFrame(todos_comentarios)

A scrapear The Father...
Botões encontrados: ['', '', '', 'All topics', '', 'Review', '', '', '', 'Spoiler', '', '', '', 'Spoiler', '', '', '', '', '', '', 'Spoiler', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Spoiler', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Spoiler', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Spoiler', '', '', '', '25 more', 'All', 'Clear all', 'Decline', 'Accept', '']
Botão 'All' encontrado na posição 100
Botão 'All' clicado para The Father
⬇️ Scroll 1/250 feito para baixo...
⬇️ Scroll 2/250 feito para baixo...
⬇️ Scroll 3/250 feito para baixo...
⬇️ Scroll 4/250 feito para baixo...
⬇️ Scroll 5/250 feito para baixo...
⬇️ Scroll 6/250 feito para baixo...
⬇️ Scroll 7/250 feito para baixo...
⬇️ Scroll 8/250 feito para baixo...
⬇️ Scroll 9/250 feito para baixo...
⬇️ Scroll 10/250 feito para baixo...
⬇️ Scroll

In [6]:
# Os spoilers que estão por baixo do botão de Spoiler nos comentários não são capturados
df_reviews

,Filme,Comentario,Data
0,The Father,"""I don't know what's happening anymore.""","Apr 13, 2021"
1,The Father,How does one respond when they cannot know rea...,"Mar 22, 2020"
2,The Father,Something I'll never forget - This is a film t...,"Sep 17, 2020"
3,The Father,So good I never want to watch it again,"Mar 14, 2021"
4,The Father,This movie will haunt me for awhile. - It's ju...,"Mar 19, 2021"
...,...,...,...
20846,The Black Phone,"I have seen ""better"" films in my life. - When ...","Jul 17, 2022"
20847,The Black Phone,Or you could use the toilet basin to smash his...,"Oct 27, 2022"
20848,The Black Phone,Nice original ideas with some small let downs,"Jul 16, 2022"
20849,The Black Phone,Pretty decent kidnap film - I loved the edit o...,"Dec 29, 2022"


In [ ]:
df_reviews.to_csv('imdb_reviews.csv', index=False)